# Time to event analysis

This tab aims to investigate the impact of the treatment on the risk of an event occurring (often tumor progression or unfortunately death of the patient).

The most common graph is the representation of Kaplan-Meier curves. An important advantage of the Kaplan–Meier curve is that the method can take into account some types of censored data. This representation is completed with a statistical test (e.g., log-rank test) for significance investigation.

In [1]:
import os
import sys
sys.path.append("C:\\Users\\Fabien Boux\\Code\\ClinLib")

from functions.config import Config
from clinlib.database import Database

with open('init.txt') as f:
    lines = f.readlines()
config = Config(lines[0])
config.read()

database = Database(config.get_value('database', section='PATH'), idlength=(int(config.get_value('id_length', section='OTHER')) if config.is_key('id_length') else 3))
database.add_resource({'metadata': os.path.join(config.get_value('database', section='PATH'), config.get_value('metadata', section='PATH'))})

import ipywidgets as widgets
from ipywidgets import interact, interactive
import matplotlib.pyplot as plt
from IPython.display import display

In [2]:
%matplotlib widget

from functions.graph import kaplan_meier_plot

if not config.is_key('followup'):
    config.extract_config_values('followup')
followup = int(int(config.get_value('followup')) / (365/12))

metric = (config.get_value('volume_label', section='OTHER') if config.is_key('volume_label') else 'Volume')
visits = (config.get_value('visits', section='OTHER') if config.is_key('visits') else None)
groups = (config.get_value('groups', section='OTHER') if config.is_key('groups') else None)


def plot_km(Event='OS', Followup=followup):
    if Event == 'PFS (adjusted)':
        fig = kaplan_meier_plot(database, event='PFS', adjust_ipfs=True, groups=groups,
                                visits=visits, followup_time=Followup, metric=metric)
    else:
        fig = kaplan_meier_plot(database, event=Event, adjust_ipfs=False, groups=groups,
                                visits=visits, followup_time=Followup, metric=metric)
    plt.show()

interact(plot_km, Event=['OS', 'PFS', 'PFS (adjusted)'], Followup=(0, round(followup*1.5), 1));

C:\ProgramData\Anaconda3\envs\NanoRad\lib\site-packages\statsmodels\compat\pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


interactive(children=(Dropdown(description='Event', options=('OS', 'PFS', 'PFS (adjusted)'), value='OS'), IntS…